In [12]:
import json
import folium
import pandas
import requests
from bs4 import BeautifulSoup
import urllib.request
import geotag
import kml2geojson 
import zipfile
from xml.dom import minidom
import os
import topojson
import geojson


# dependencies: topojson, geojson, shapely, geopandas

In [13]:
from shapely.geometry import shape, Point, MultiPoint
import math
import json


def get_bounds(geometries):
    """Returns bounding box of geometries. Implementation creates a MultiPoint
    from the boxes of all polygons in order to get the result"""
    points = []
    for g in geometries:
        bounds = g.bounds
        points.extend([Point(bounds[:2]), Point(bounds[2:])])
    return MultiPoint(points).bounds


def lines2arcs(geometries, kx, ky, x0, y0):
    """Naive approach to create arcs from lines. TODO: implement detection of
    shared points/arcs"""
    for g in geometries:
        try:
            boundary = g.boundary
        except AttributeError:
            continue

        if boundary.type == 'LineString':
            boundary = [boundary]

        arcs = []
        x = y = 0
        for line in boundary:
            arc = []
            for a, b in line.coords:
                a = int(round((a - x0) / kx - x))
                b = int(round((b - y0) / ky - y))
                x += a
                y += b
                arc.append([a, b])
            arcs.append(arc)
        return arcs


def geo2topo(geojson, quant_factor=1e4):
    """Given a geojson dict, returns topojson"""
    assert geojson['type'] == "FeatureCollection"

    geometries = []

    for feature in geojson['features']:
        geometries.append(shape(feature['geometry']))

    x0, y0, x1, y1 = get_bounds(geometries)
    kx, ky = 1 / ((quant_factor - 1) / (x1 - x0)), 1/ ((quant_factor - 1) / (y1 - y0))

    arcs = lines2arcs(geometries, kx, ky, x0, y0)

    return {
        "type" : "Topology",
        "transform" : {
  	  "scale" : [kx, ky],
		  "translate" : [x0, y0]
	    },
        "objects" : {},
        "arcs" : arcs
    }


In [17]:
topic = "171384"
place_file = open("initial-10/places_"+topic+".json")
places = json.load(place_file)
path = ("./topics/" + topic)
#os.mkdir(path)
#os.mkdir(path + "/shp-zipfiles")
#os.mkdir(path + "/shp-files")
#os.mkdir(path + "/geojsons")

# soup.
# https://gadm.org/maps.html

r = requests.get("https://gadm.org/maps.html")
htmlcontent = BeautifulSoup(r.content, "html.parser")
countries = []
country_containers = htmlcontent.find('div', class_="container-fluid main-container").p.small
# print(country_containers)

country_codes = {}
count = 'A'

for a_tag in country_containers.find_all('a', href=True):
    name = a_tag.string
    if name[0].islower():
        name = count + name
        count = chr(ord(count) + 1)
        if (count == 'X'):
            count = chr(ord(count) + 1)
    country_codes.update({name: (a_tag['href'][5:8])})


In [24]:
def get_shp(code):
    url="https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_" + code + "_shp.zip"
   #     https://biogeo.ucdavis.edu/data/gadm3.6/shp/gadm36_ALB_shp.zip
    
    urllib.request.urlretrieve(url, path + "/shp-zipfiles/" + code + "_shp.zip")
    print(code)
    zf = zipfile.ZipFile(path + "/shp-zipfiles/" + code + "_shp.zip")
    
    zf.extractall(path + "/shp-files/")
    zf.close()
   # kml2geojson.main.convert(path + '/shp-files/gadm36_' + code + "_0.shp", path + "/geojsons/", separate_folders=False, style_type=None, style_filename='style.json')
    
    
    
    

In [25]:
# https://biogeo.ucdavis.edu/data/gadm3.6/kmz/gadm36_AFG_0.kmz
GT = geotag.Geotag()
place_names = []

for place in places:
    tag = place["geotag"]
    if (tag in GT.path2entry):
        tag = GT.path2entry[tag]
        place_names.append(tag['name'])

print(place_names)

for place in place_names:
    if (place in country_codes.keys()):
        country = country_codes.get(place)
        get_shp(country)

        
            

['Vienna', 'Austria', 'Germany', 'Neuss', 'Berlin', 'Graz', 'Duesseldorf', 'France', 'Syria', 'Albania', 'Belgium', 'North Rhine-Westphalia', 'Tunisia', 'Paris', 'Salzburg']
AUT
DEU
FRA
SYR
ALB
BEL
TUN
